In [1]:
#Imports
import json
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openweathermapy.core as ow
from citipy import citipy 
import random
import requests as req

#OpenWeatherMap API Key
api_key = "26fea83d1259ec5c651394906a953b54"
base_url = "http://api.openweathermap.org/data/2.5/weather"


In [2]:
#Random selection of 500 unique cities based on lat/long coordinates
#Truncate random coordinates to 6 decimal points each
def truncate(x, d): #Source for truncate() function: https://stackoverflow.com/a/20545024
    return int(x*(10.0**d))/(10.0**d)
    #x = input
    #d = decimal places to truncate to

cityList = []
selected_cites = []
uniqueCityCount = 0

while (uniqueCityCount < 10): 
    random_lat = truncate(random.uniform(-90,90),6)
    random_long = truncate(random.uniform(-180,180),6)
    cityToCheck = citipy.nearest_city(random_lat, random_long).city_name
    #Pass query to OpenWeatherMap API to check if city is in database
    
    query_city = cityToCheck
    query_url = base_url + "?apikey=" + api_key + "&q=" + query_city
    response = req.get(query_url)
    json_data = response.json()
    
    #Prevent duplicates from being appended to the cityList (before dataframe creation)
    #While also not adding the specified city if not in OWM database
    if cityToCheck not in cityList and json_data['cod'] != '404':
        cityList.append(cityToCheck)
        uniqueCityCount = uniqueCityCount+ 1 
        
#Create dataframe with uniqueCity names        
raw_cities_df = pd.DataFrame({"City Name": cityList})

#Not needed if duplicates removed before dataframe creation: 
#raw_cities_df = raw_cities_df.drop_duplicates() 

#Creation of column for city number
raw_cities_df['City Number'] = raw_cities_df.index + 1
raw_cities_df= raw_cities_df[['City Number','City Name']]
cn = pd.to_numeric(raw_cities_df['City Number'], errors='raise', downcast=None)

In [3]:

raw_cities_df.head(25)
#print(raw_cities_df['City Number'])

,City Number,City Name
0,1,hobart
1,2,vaitape
2,3,yellowknife
3,4,yar-sale
4,5,cape town
5,6,medzilaborce
6,7,bredasdorp
7,8,busselton
8,9,karratha
9,10,ibotirama


In [4]:
settings = {"units": "imperial","appid": api_key}
from time import sleep

#Print parameters being requested from owm API before retrival
print("Base URL for API requests:... " + ow.BASE_URL)
print("Parameters being requested per city:... ")
summary = ["name", "coord.lat", "coord.lon","main.temp","main.humidity","wind.speed","clouds.all"]
print(summary)

#Get current weather for each city in cityList
cityCount = 1
for city in raw_cities_df["City Name"]:
    sleep(1.2)
    try:
        print("Processing city " + str(cityCount)+ " of " +str(len(raw_cities_df["City Name"])) + ", "+ city +":") 
        current_city= ow.get_current(city, **settings)
        sleep(1.2)
        #Leveraging OpenWatherMapy to parse the response with current_city(*summary)
        #summary = ["name", "coord.lat", "coord.lon","main.temp","main.humidity","wind.speed","clouds.all"]
        summary_output = current_city(*summary)
        print(str(summary_output))
    
    except Exception as ex:
        print(ex)
    cityCount = cityCount + 1

    #openweathermap complete city list:
    #http://openweathermap.org/help/city_list.txt 
    #one method ---> check against list
    #another method ---> skip if error returned 
    
    #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_html.html
    
    

Base URL for API requests:... http://api.openweathermap.org/data/2.5/
Parameters being requested per city:... 
['name', 'coord.lat', 'coord.lon', 'main.temp', 'main.humidity', 'wind.speed', 'clouds.all']
Processing city 1 of 10, hobart:
('Hobart', -42.88, 147.33, 73.4, 38, 11.41, 92)
Processing city 2 of 10, vaitape:
('Vaitape', -16.52, -151.75, 78.78, 100, 10.22, 80)
Processing city 3 of 10, yellowknife:
('Yellowknife', 62.45, -114.38, 8.6, 84, 8.05, 90)
Processing city 4 of 10, yar-sale:
('Yar-Sale', 66.83, 70.83, 25.55, 95, 12.24, 76)
Processing city 5 of 10, cape town:
('Cape Town', -33.93, 18.42, 68, 77, 12.75, 75)
Processing city 6 of 10, medzilaborce:
('Medzilaborce', 49.27, 21.9, 36.08, 100, 5.75, 92)
Processing city 7 of 10, bredasdorp:
('Bredasdorp', -34.53, 20.04, 60.8, 87, 3.36, 64)
Processing city 8 of 10, busselton:
('Busselton', -33.64, 115.35, 62.94, 100, 10.22, 0)
Processing city 9 of 10, karratha:
('Karratha', -20.74, 116.85, 84, 78, 5.75, 0)
Processing city 10 of 10,